# METU Ring Bus Survey - Exploratory Data Analysis
## STAT365 Sampling and Survey Techniques

This notebook contains the exploratory data analysis for the METU Campus Ring Bus Usage and Satisfaction Survey.

## Setup and Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

# Set style for better visuals
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# For inline display in notebooks
%matplotlib inline

# Increase default figure size
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Load data
# For Google Colab: Upload the file or mount Google Drive
# from google.colab import files
# uploaded = files.upload()

df = pd.read_csv('metu_ring_processed.csv')

print(f"Dataset Shape: {df.shape[0]} respondents, {df.shape[1]} variables")
print(f"\nColumns: {list(df.columns)}")

In [ ]:
# Quick look at the data
df.head()

In [ ]:
# Data types and missing values
df.info()

---
## Section 1: Sample Description & Demographics

In [ ]:
fig = plt.figure(figsize=(16, 12))
gs = GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)

# 1.1 Gender Distribution
ax1 = fig.add_subplot(gs[0, 0])
gender_counts = df['Gender'].value_counts()
colors = ['#3498db', '#e74c3c', '#2ecc71']
wedges, texts, autotexts = ax1.pie(gender_counts, labels=gender_counts.index, 
                                    autopct='%1.1f%%', colors=colors[:len(gender_counts)],
                                    explode=[0.02]*len(gender_counts))
ax1.set_title('Gender Distribution', fontsize=12, fontweight='bold')

# 1.2 Residence Distribution
ax2 = fig.add_subplot(gs[0, 1])
residence_counts = df['Residence_Simplified'].value_counts()
bars = ax2.barh(residence_counts.index, residence_counts.values, color='#3498db')
ax2.set_xlabel('Count')
ax2.set_title('Residence Distribution', fontsize=12, fontweight='bold')
for i, v in enumerate(residence_counts.values):
    ax2.text(v + 1, i, f'{v} ({v/len(df)*100:.1f}%)', va='center', fontsize=9)
ax2.set_xlim(0, max(residence_counts.values) * 1.3)

# 1.3 Study Level Distribution
ax3 = fig.add_subplot(gs[0, 2])
study_counts = df['Study_Level'].value_counts()
bars = ax3.bar(range(len(study_counts)), study_counts.values, color='#2ecc71')
ax3.set_xticks(range(len(study_counts)))
ax3.set_xticklabels([s[:15] + '...' if len(s) > 15 else s for s in study_counts.index], 
                    rotation=45, ha='right', fontsize=9)
ax3.set_ylabel('Count')
ax3.set_title('Study Level Distribution', fontsize=12, fontweight='bold')
for i, v in enumerate(study_counts.values):
    ax3.text(i, v + 2, f'{v}', ha='center', fontsize=9)

# 1.4 Faculty Distribution
ax4 = fig.add_subplot(gs[1, :2])
faculty_counts = df['Faculty'].value_counts()
colors_faculty = plt.cm.Set3(np.linspace(0, 1, len(faculty_counts)))
bars = ax4.barh(range(len(faculty_counts)), faculty_counts.values, color=colors_faculty)
ax4.set_yticks(range(len(faculty_counts)))
ax4.set_yticklabels([f[:35] + '...' if len(f) > 35 else f for f in faculty_counts.index], fontsize=9)
ax4.set_xlabel('Count')
ax4.set_title('Faculty Distribution', fontsize=12, fontweight='bold')
for i, v in enumerate(faculty_counts.values):
    ax4.text(v + 1, i, f'{v} ({v/len(df)*100:.1f}%)', va='center', fontsize=9)
ax4.set_xlim(0, max(faculty_counts.values) * 1.25)
ax4.invert_yaxis()

# 1.5 Top 10 Departments
ax5 = fig.add_subplot(gs[1, 2])
dept_counts = df['Department'].value_counts().head(10)
bars = ax5.barh(range(len(dept_counts)), dept_counts.values, color='#9b59b6')
ax5.set_yticks(range(len(dept_counts)))
ax5.set_yticklabels([d[:20] + '...' if len(d) > 20 else d for d in dept_counts.index], fontsize=8)
ax5.set_xlabel('Count')
ax5.set_title('Top 10 Departments', fontsize=12, fontweight='bold')
ax5.invert_yaxis()

plt.suptitle('Section 1: Sample Demographics', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Demographic Summary Table
print("=" * 50)
print("DEMOGRAPHIC SUMMARY")
print("=" * 50)

print("\nGender:")
for g, c in gender_counts.items():
    print(f"  {g}: {c} ({c/len(df)*100:.1f}%)")

print("\nResidence:")
for r, c in residence_counts.items():
    print(f"  {r}: {c} ({c/len(df)*100:.1f}%)")

print("\nStudy Level:")
for s, c in study_counts.items():
    print(f"  {s}: {c} ({c/len(df)*100:.1f}%)")

---
## Section 2: Transportation Mode Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 2.1 Transportation Mode Frequency (Mean Scores)
transport_cols = ['Freq_Taxi', 'Freq_Ring', 'Freq_Walking', 'Freq_Hitchhiking', 'Freq_PersonalVehicle']
transport_labels = ['Taxi', 'Ring Bus', 'Walking', 'Hitchhiking', 'Personal Vehicle']
transport_means = df[transport_cols].mean()
transport_std = df[transport_cols].std()

colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']
bars = axes[0].bar(transport_labels, transport_means, yerr=transport_std, 
                   color=colors, capsize=5, alpha=0.8)
axes[0].set_ylabel('Mean Frequency Score (1-5)')
axes[0].set_title('Transportation Mode Usage Frequency', fontsize=12, fontweight='bold')
axes[0].set_ylim(0, 5.5)
for i, (m, s) in enumerate(zip(transport_means, transport_std)):
    axes[0].text(i, m + s + 0.2, f'{m:.2f}', ha='center', fontsize=10)

# 2.2 Ring Bus Usage Distribution
ring_freq = df['Freq_Ring'].value_counts().sort_index()
freq_labels = ['Never (1)', 'Rarely (2)', 'Sometimes (3)', 'Often (4)', 'Always (5)']
axes[1].bar(freq_labels, [ring_freq.get(i, 0) for i in range(1, 6)], color='#3498db')
axes[1].set_ylabel('Count')
axes[1].set_title('Ring Bus Usage Frequency Distribution', fontsize=12, fontweight='bold')
axes[1].tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show()

In [ ]:
print("Transportation Mode Mean Frequency (1=Never, 5=Always)")
print("-" * 45)
for label, mean, std in zip(transport_labels, transport_means, transport_std):
    print(f"  {label:20s}: {mean:.2f} ± {std:.2f}")

---
## Section 3: Ring Bus Usage Patterns

In [ ]:
fig = plt.figure(figsize=(16, 14))
gs = GridSpec(3, 2, figure=fig, hspace=0.35, wspace=0.25)

# 3.1 Weekday vs Weekend Usage Frequency
ax1 = fig.add_subplot(gs[0, 0])
weekday_freq = df['Weekday_Ring_Frequency'].value_counts().sort_index()
weekend_freq = df['Weekend_Ring_Frequency'].value_counts().sort_index()

x = np.arange(6)
width = 0.35
weekday_vals = [weekday_freq.get(i, 0) for i in range(6)]
weekend_vals = [weekend_freq.get(i, 0) for i in range(6)]

bars1 = ax1.bar(x - width/2, weekday_vals, width, label='Weekday', color='#3498db')
bars2 = ax1.bar(x + width/2, weekend_vals, width, label='Weekend', color='#e74c3c')
ax1.set_xlabel('Frequency Level')
ax1.set_ylabel('Count')
ax1.set_title('Weekday vs Weekend Ring Bus Usage', fontsize=12, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(['Never', 'Very Rarely', '1-2/week', '3-4/week', 'Once/day', '>Once/day'])
ax1.tick_params(axis='x', rotation=30)
ax1.legend()

# 3.2 Peak Usage Times
ax2 = fig.add_subplot(gs[0, 1])
time_cols_wkday = ['TimeWkday_Morning_08:00_11:00', 'TimeWkday_Midday_11:00_15:00', 
                   'TimeWkday_Afternoon_15:00_19:00', 'TimeWkday_Late_19:00_23:00']
time_cols_wkend = ['TimeWkend_Morning_08:00_11:00', 'TimeWkend_Midday_11:00_15:00',
                   'TimeWkend_Afternoon_15:00_19:00', 'TimeWkend_Late_19:00_23:00']
time_labels = ['Morning\n(08-11)', 'Midday\n(11-15)', 'Afternoon\n(15-19)', 'Late\n(19-23)']

wkday_times = df[time_cols_wkday].sum()
wkend_times = df[time_cols_wkend].sum()

x = np.arange(4)
bars1 = ax2.bar(x - width/2, wkday_times.values, width, label='Weekday', color='#3498db')
bars2 = ax2.bar(x + width/2, wkend_times.values, width, label='Weekend', color='#e74c3c')
ax2.set_xlabel('Time Slot')
ax2.set_ylabel('Number of Users')
ax2.set_title('Peak Usage Times', fontsize=12, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(time_labels)
ax2.legend()

# 3.3 Route Popularity
ax3 = fig.add_subplot(gs[1, 0])
route_cols = ['Route_Brown_A1___A1', 'Route_Turquoise_West___A2', 'Route_Orange_West___A2',
              'Route_Yellow_A2___East', 'Route_Red_East___A2', 'Route_Navy_Blue_East___East',
              'Route_Purple_A1___A1']
route_labels = ['Brown\n(A1-A1)', 'Turquoise\n(West-A2)', 'Orange\n(West-A2)', 
                'Yellow\n(A2-East)', 'Red\n(East-A2)', 'Navy Blue\n(East-East)', 'Purple\n(A1-A1)']
route_colors = ['#8B4513', '#40E0D0', '#FFA500', '#FFD700', '#FF0000', '#000080', '#800080']

route_usage = df[route_cols].sum()
bars = ax3.bar(route_labels, route_usage.values, color=route_colors)
ax3.set_ylabel('Number of Users')
ax3.set_title('Route Popularity', fontsize=12, fontweight='bold')
for i, v in enumerate(route_usage.values):
    ax3.text(i, v + 2, str(v), ha='center', fontsize=9)

# 3.4 Purpose of Using Ring Bus
ax4 = fig.add_subplot(gs[1, 1])
purpose_cols = ['Purpose_Going_to_A1_Gate', 'Purpose_Going_to_A2_Gate', 'Purpose_Going_to_department',
                'Purpose_Going_to_East_Dorms', 'Purpose_Going_to_West_Dorms', 'Purpose_Going_to_cafeteria',
                'Purpose_Going_to_library', 'Purpose_Going_to_sports_facilitie']
purpose_labels = ['A1 Gate', 'A2 Gate', 'Department', 'East Dorms', 'West Dorms', 
                  'Cafeteria', 'Library', 'Sports']
purpose_usage = df[purpose_cols].sum().sort_values(ascending=True)
purpose_labels_sorted = [purpose_labels[purpose_cols.index(c)] for c in purpose_usage.index]

bars = ax4.barh(purpose_labels_sorted, purpose_usage.values, color='#2ecc71')
ax4.set_xlabel('Number of Users')
ax4.set_title('Purpose of Using Ring Bus', fontsize=12, fontweight='bold')
for i, v in enumerate(purpose_usage.values):
    ax4.text(v + 1, i, str(v), va='center', fontsize=9)

# 3.5 Reasons for Using Ring Bus
ax5 = fig.add_subplot(gs[2, 0])
reason_cols = ['Reason_Free', 'Reason_No_private_car', 'Reason_Most_convenient',
               'Reason_Fastest', 'Reason_Better_for_environment', 'Reason_Hard_to_find_parking']
reason_labels = ['Free', 'No Private Car', 'Most Convenient', 'Fastest', 
                 'Better for Environment', 'Hard to Find Parking']
reason_usage = df[reason_cols].sum().sort_values(ascending=True)
reason_labels_sorted = [reason_labels[reason_cols.index(c)] for c in reason_usage.index]

bars = ax5.barh(reason_labels_sorted, reason_usage.values, color='#9b59b6')
ax5.set_xlabel('Number of Users')
ax5.set_title('Reasons for Using Ring Bus', fontsize=12, fontweight='bold')
for i, v in enumerate(reason_usage.values):
    ax5.text(v + 1, i, str(v), va='center', fontsize=9)

# 3.6 Average Wait Time Distribution
ax6 = fig.add_subplot(gs[2, 1])
wait_time = df['Avg_Wait_Time'].value_counts().sort_index()
wait_labels = ['<5 min', '5-10 min', '11-15 min', '16-20 min', '>20 min']
wait_vals = [wait_time.get(i, 0) for i in range(1, 6)]
bars = ax6.bar(wait_labels, wait_vals, color='#f39c12')
ax6.set_ylabel('Count')
ax6.set_title('Average Wait Time Distribution', fontsize=12, fontweight='bold')
for i, v in enumerate(wait_vals):
    ax6.text(i, v + 2, f'{v}\n({v/len(df)*100:.1f}%)', ha='center', fontsize=9)

plt.suptitle('Section 3: Ring Bus Usage Patterns', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

---
## Section 4: Satisfaction Analysis

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)

# 4.1 Overall Satisfaction Distribution
ax1 = fig.add_subplot(gs[0, 0])
overall_sat = df['Overall_Satisfaction'].value_counts().sort_index()
colors_sat = ['#e74c3c', '#f39c12', '#f1c40f', '#2ecc71', '#27ae60']
bars = ax1.bar([1, 2, 3, 4, 5], [overall_sat.get(i, 0) for i in range(1, 6)], color=colors_sat)
ax1.set_xlabel('Satisfaction Score')
ax1.set_ylabel('Count')
ax1.set_title('Overall Satisfaction Distribution', fontsize=12, fontweight='bold')
ax1.set_xticks([1, 2, 3, 4, 5])
ax1.set_xticklabels(['Very\nDissatisfied', 'Dissatisfied', 'Neutral', 'Satisfied', 'Very\nSatisfied'])
mean_sat = df['Overall_Satisfaction'].mean()
ax1.text(4.5, max(overall_sat.values)*0.9, f'Mean: {mean_sat:.2f}', fontsize=10, 
         bbox=dict(boxstyle='round', facecolor='wheat'))

# 4.2 Weekday vs Weekend Satisfaction Comparison
ax2 = fig.add_subplot(gs[0, 1])
weekday_sat = df['Weekday_Schedule_Satisfaction'].value_counts().sort_index()
weekend_sat = df['Weekend_Schedule_Satisfaction'].value_counts().sort_index()

x = np.arange(1, 6)
width = 0.35
bars1 = ax2.bar(x - width/2, [weekday_sat.get(i, 0) for i in range(1, 6)], width, 
                label='Weekday', color='#3498db')
bars2 = ax2.bar(x + width/2, [weekend_sat.get(i, 0) for i in range(1, 6)], width, 
                label='Weekend', color='#e74c3c')
ax2.set_xlabel('Satisfaction Score')
ax2.set_ylabel('Count')
ax2.set_title('Weekday vs Weekend Schedule Satisfaction', fontsize=12, fontweight='bold')
ax2.set_xticks(x)
ax2.legend()
wkday_mean = df['Weekday_Schedule_Satisfaction'].mean()
wkend_mean = df['Weekend_Schedule_Satisfaction'].mean()
ax2.text(0.02, 0.98, f'Weekday Mean: {wkday_mean:.2f}\nWeekend Mean: {wkend_mean:.2f}', 
         transform=ax2.transAxes, fontsize=9, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat'))

# 4.3 App Helpfulness
ax3 = fig.add_subplot(gs[0, 2])
app_help = df['App_Helping_Ring'].dropna().value_counts().sort_index()
bars = ax3.bar([1, 2, 3, 4, 5], [app_help.get(i, 0) for i in range(1, 6)], color='#1abc9c')
ax3.set_xlabel('Helpfulness Score')
ax3.set_ylabel('Count')
ax3.set_title('METU App Helpfulness for Ring Bus', fontsize=12, fontweight='bold')
ax3.set_xticks([1, 2, 3, 4, 5])
ax3.set_xticklabels(['Not\nHelpful', '2', '3', '4', 'Very\nHelpful'])
app_mean = df['App_Helping_Ring'].mean()
ax3.text(4, max(app_help.values)*0.9, f'Mean: {app_mean:.2f}', fontsize=10,
         bbox=dict(boxstyle='round', facecolor='wheat'))

# 4.4 Satisfaction by Residence
ax4 = fig.add_subplot(gs[1, 0])
sat_by_residence = df.groupby('Residence_Simplified')['Overall_Satisfaction'].mean().sort_values()
bars = ax4.barh(sat_by_residence.index, sat_by_residence.values, color='#3498db')
ax4.set_xlabel('Mean Satisfaction Score')
ax4.set_title('Satisfaction by Residence', fontsize=12, fontweight='bold')
ax4.set_xlim(0, 5)
for i, v in enumerate(sat_by_residence.values):
    ax4.text(v + 0.05, i, f'{v:.2f}', va='center', fontsize=9)

# 4.5 Understanding Routes Distribution
ax5 = fig.add_subplot(gs[1, 1])
understanding = df['Understanding_Routes'].dropna().value_counts().sort_index()
bars = ax5.bar([1, 2, 3, 4, 5], [understanding.get(i, 0) for i in range(1, 6)], color='#8e44ad')
ax5.set_xlabel('Ease of Understanding (1=Very Difficult, 5=Very Easy)')
ax5.set_ylabel('Count')
ax5.set_title('Ease of Understanding Routes', fontsize=12, fontweight='bold')
ax5.set_xticks([1, 2, 3, 4, 5])
understand_mean = df['Understanding_Routes'].mean()
ax5.text(4, max(understanding.values)*0.9, f'Mean: {understand_mean:.2f}', fontsize=10,
         bbox=dict(boxstyle='round', facecolor='wheat'))

# 4.6 Would Use More If Frequent
ax6 = fig.add_subplot(gs[1, 2])
use_more = df['UseMore_IfFrequent'].value_counts()
colors_pie = ['#2ecc71', '#e74c3c', '#f39c12']
wedges, texts, autotexts = ax6.pie(use_more, labels=use_more.index, autopct='%1.1f%%',
                                    colors=colors_pie[:len(use_more)], explode=[0.02]*len(use_more))
ax6.set_title('Would Use Ring More If Frequent?', fontsize=12, fontweight='bold')

plt.suptitle('Section 4: Satisfaction Analysis', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
print("=" * 50)
print("SATISFACTION SUMMARY")
print("=" * 50)
print(f"\nOverall Satisfaction: Mean = {mean_sat:.2f}, Median = {df['Overall_Satisfaction'].median()}")
print(f"Weekday Schedule Satisfaction: Mean = {wkday_mean:.2f}")
print(f"Weekend Schedule Satisfaction: Mean = {wkend_mean:.2f}")
print(f"App Helpfulness: Mean = {app_mean:.2f}")
print(f"Understanding Routes: Mean = {understand_mean:.2f}")

print("\nWould Use More If More Frequent:")
for response, count in use_more.items():
    print(f"  {response}: {count} ({count/len(df)*100:.1f}%)")

---
## Section 5: Problem Analysis

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)

# 5.1 Problem Frequency (Mean Scores)
ax1 = fig.add_subplot(gs[0, :])
problem_cols = ['Problem_LateBus', 'Problem_TooCrowded', 'Problem_LongWait', 'Problem_NotMyRoute',
                'Problem_RouteConfusing', 'Problem_HoursMismatch', 'Problem_NotClean', 'Problem_DriverBehavior']
problem_labels = ['Late Bus', 'Too Crowded', 'Long Wait', 'Not My Route', 
                  'Route Confusing', 'Hours Mismatch', 'Not Clean', 'Driver Behavior']
problem_means = df[problem_cols].mean().sort_values(ascending=False)
problem_std = df[problem_cols].std()

colors_prob = plt.cm.Reds(np.linspace(0.3, 0.9, len(problem_means)))
problem_labels_sorted = [problem_labels[problem_cols.index(c)] for c in problem_means.index]

bars = ax1.barh(range(len(problem_means)), problem_means.values, 
                xerr=[problem_std[c] for c in problem_means.index],
                color=colors_prob, capsize=3)
ax1.set_yticks(range(len(problem_means)))
ax1.set_yticklabels(problem_labels_sorted)
ax1.set_xlabel('Mean Problem Frequency (1=Never, 5=Always)')
ax1.set_title('Problem Frequency Ranking', fontsize=12, fontweight='bold')
ax1.set_xlim(0, 5.5)
for i, v in enumerate(problem_means.values):
    ax1.text(v + 0.15, i, f'{v:.2f}', va='center', fontsize=9)
ax1.invert_yaxis()

# 5.2 Problem Severity Index Distribution
ax2 = fig.add_subplot(gs[1, 0])
ax2.hist(df['Problem_Severity_Index'], bins=20, color='#e74c3c', edgecolor='black', alpha=0.7)
ax2.axvline(df['Problem_Severity_Index'].mean(), color='blue', linestyle='--', 
            label=f'Mean: {df["Problem_Severity_Index"].mean():.2f}')
ax2.axvline(df['Problem_Severity_Index'].median(), color='green', linestyle='--',
            label=f'Median: {df["Problem_Severity_Index"].median():.2f}')
ax2.set_xlabel('Problem Severity Index')
ax2.set_ylabel('Count')
ax2.set_title('Problem Severity Index Distribution', fontsize=12, fontweight='bold')
ax2.legend()

# 5.3 Most Valuable Improvement
ax3 = fig.add_subplot(gs[1, 1])
improvement = df['MostValuable_Improvement'].value_counts()
colors_imp = plt.cm.Set2(np.linspace(0, 1, len(improvement)))
bars = ax3.barh(range(len(improvement)), improvement.values, color=colors_imp)
ax3.set_yticks(range(len(improvement)))
ax3.set_yticklabels(improvement.index, fontsize=9)
ax3.set_xlabel('Count')
ax3.set_title('Most Valuable Improvement Requested', fontsize=12, fontweight='bold')
for i, v in enumerate(improvement.values):
    ax3.text(v + 1, i, f'{v} ({v/len(df)*100:.1f}%)', va='center', fontsize=9)
ax3.invert_yaxis()

plt.suptitle('Section 5: Problem Analysis', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
print("=" * 50)
print("PROBLEM ANALYSIS SUMMARY")
print("=" * 50)

print("\nProblem Frequency (Mean Score, 1=Never, 5=Always):")
for label, mean in zip(problem_labels_sorted, problem_means.values):
    print(f"  {label:20s}: {mean:.2f}")

print(f"\nProblem Severity Index:")
print(f"  Mean: {df['Problem_Severity_Index'].mean():.2f}")
print(f"  Median: {df['Problem_Severity_Index'].median():.2f}")
print(f"  Std: {df['Problem_Severity_Index'].std():.2f}")

print("\nMost Valuable Improvement:")
for imp, count in improvement.items():
    print(f"  {imp}: {count} ({count/len(df)*100:.1f}%)")

---
## Section 6: User Segmentation (Heavy vs Light Users)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 6.1 Heavy User Distribution
heavy_counts = df['Heavy_User'].value_counts()
labels = ['Light Users', 'Heavy Users']
colors = ['#3498db', '#e74c3c']
axes[0, 0].pie(heavy_counts, labels=labels, autopct='%1.1f%%', colors=colors, explode=[0.02, 0.02])
axes[0, 0].set_title('Heavy vs Light Users Distribution', fontsize=12, fontweight='bold')

# 6.2 Satisfaction Comparison by User Type
ax = axes[0, 1]
heavy_sat = df.groupby('Heavy_User')['Overall_Satisfaction'].mean()
bars = ax.bar(['Light Users', 'Heavy Users'], heavy_sat.values, color=['#3498db', '#e74c3c'])
ax.set_ylabel('Mean Overall Satisfaction')
ax.set_title('Satisfaction by User Type', fontsize=12, fontweight='bold')
ax.set_ylim(0, 5)
for i, v in enumerate(heavy_sat.values):
    ax.text(i, v + 0.1, f'{v:.2f}', ha='center', fontsize=10)

# 6.3 Problem Severity by User Type
ax = axes[1, 0]
heavy_prob = df.groupby('Heavy_User')['Problem_Severity_Index'].mean()
bars = ax.bar(['Light Users', 'Heavy Users'], heavy_prob.values, color=['#3498db', '#e74c3c'])
ax.set_ylabel('Mean Problem Severity Index')
ax.set_title('Problem Severity by User Type', fontsize=12, fontweight='bold')
for i, v in enumerate(heavy_prob.values):
    ax.text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10)

# 6.4 Residence Distribution by User Type
ax = axes[1, 1]
cross_tab = pd.crosstab(df['Residence_Simplified'], df['Heavy_User'], normalize='columns') * 100
cross_tab.plot(kind='bar', ax=ax, color=['#3498db', '#e74c3c'])
ax.set_ylabel('Percentage')
ax.set_title('Residence Distribution by User Type', fontsize=12, fontweight='bold')
ax.legend(['Light Users', 'Heavy Users'])
ax.tick_params(axis='x', rotation=45)

plt.suptitle('Section 6: User Segmentation', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
print("=" * 50)
print("USER SEGMENTATION SUMMARY")
print("=" * 50)

print(f"\nHeavy User Distribution:")
print(f"  Heavy Users: {heavy_counts.get(1, 0)} ({heavy_counts.get(1, 0)/len(df)*100:.1f}%)")
print(f"  Light Users: {heavy_counts.get(0, 0)} ({heavy_counts.get(0, 0)/len(df)*100:.1f}%)")

print(f"\nComparison: Heavy vs Light Users:")
print(f"  Overall Satisfaction - Heavy: {heavy_sat.get(1, 0):.2f}, Light: {heavy_sat.get(0, 0):.2f}")
print(f"  Problem Severity - Heavy: {heavy_prob.get(1, 0):.2f}, Light: {heavy_prob.get(0, 0):.2f}")

---
## Section 7: Correlation Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 7.1 Correlation Heatmap for Key Variables
key_vars = ['Overall_Satisfaction', 'Weekday_Schedule_Satisfaction', 'Weekend_Schedule_Satisfaction',
            'Understanding_Routes', 'App_Helping_Ring', 'Problem_Severity_Index', 
            'Total_Usage_Score', 'Avg_Wait_Time']
corr_matrix = df[key_vars].corr()

sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdYlBu_r', center=0,
            ax=axes[0], square=True, linewidths=0.5)
axes[0].set_title('Correlation Matrix: Key Variables', fontsize=12, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)
axes[0].tick_params(axis='y', rotation=0)

# 7.2 Satisfaction vs Problem Severity Scatter
ax = axes[1]
scatter = ax.scatter(df['Problem_Severity_Index'], df['Overall_Satisfaction'], 
                     c=df['Heavy_User'], cmap='RdYlBu', alpha=0.6, edgecolors='black', linewidth=0.5)
ax.set_xlabel('Problem Severity Index')
ax.set_ylabel('Overall Satisfaction')
ax.set_title('Satisfaction vs Problem Severity', fontsize=12, fontweight='bold')

# Add trend line
z = np.polyfit(df['Problem_Severity_Index'].dropna(), 
               df.loc[df['Problem_Severity_Index'].notna(), 'Overall_Satisfaction'], 1)
p = np.poly1d(z)
x_line = np.linspace(df['Problem_Severity_Index'].min(), df['Problem_Severity_Index'].max(), 100)
ax.plot(x_line, p(x_line), "r--", alpha=0.8, 
        label=f'Trend (r={corr_matrix.loc["Overall_Satisfaction", "Problem_Severity_Index"]:.2f})')
ax.legend()
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('User Type (0=Light, 1=Heavy)')

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 50)
print("KEY CORRELATIONS WITH OVERALL SATISFACTION")
print("=" * 50)

sat_corrs = corr_matrix['Overall_Satisfaction'].drop('Overall_Satisfaction').sort_values(key=abs, ascending=False)
for var, corr in sat_corrs.items():
    print(f"  {var:35s}: r = {corr:+.3f}")

---
## Summary Statistics

In [ ]:
# Create comprehensive summary table
summary_vars = ['Overall_Satisfaction', 'Weekday_Schedule_Satisfaction', 'Weekend_Schedule_Satisfaction',
                'Understanding_Routes', 'App_Helping_Ring', 'Problem_Severity_Index', 
                'Total_Usage_Score', 'Avg_Wait_Time']
summary_stats = df[summary_vars].describe().T
summary_stats['median'] = df[summary_vars].median()
summary_stats = summary_stats[['count', 'mean', 'std', 'median', 'min', 'max']]
summary_stats

---
## Key Findings

### Sample
- 325 respondents, predominantly undergraduates (89.8%)
- Majority from East Dorms (42.5%) and West Dorms (35.4%)
- Engineering faculty dominates the sample

### Usage Patterns
- Ring Bus is heavily used (mean = 3.96/5)
- Yellow (A2-East) is the most popular route (64%)
- Peak usage: Midday and Afternoon on weekdays
- Most users wait 5-15 minutes

### Satisfaction Concerns
- Overall satisfaction is mediocre (mean = 2.96)
- **Weekend satisfaction is notably low** (mean = 1.93)
- 89.8% would use Ring more if buses were more frequent

### Main Problems
1. Too Crowded (4.05/5)
2. Long Wait (3.78/5)
3. Not My Route (3.10/5)

### Improvement Priorities
1. More frequent buses (50.5%)
2. Better route coverage (20.3%)
3. Extended service hours (15.7%)